In [191]:
from collections import defaultdict
from functools import lru_cache
import random
import requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statbotics

%matplotlib notebook

sb = statbotics.Statbotics()

In [10]:
all_teams = sb.get_team_years(year=2023, limit=10000)
all_teams_dict = {t["team"]: t for t in all_teams}
len(all_teams)

3286

In [6]:
champ_teams = sb.get_team_events(event="2023cmptx", limit=10000)
champ_team_nums = [t["team"] for t in champ_teams]
champ_team_nums += [333, 449, 999, 2170, 2337, 4779, 4976, 5123, 7174, 7534, 8013, 9152, 9244]

print(len(set(champ_team_nums)))

620


In [51]:
team_to_epa = {t: all_teams_dict[t]["epa_end"] for t in champ_team_nums}
team_to_rp_1_epa = {t: all_teams_dict[t]["rp_1_epa_end"] for t in champ_team_nums}
team_to_rp_2_epa = {t: all_teams_dict[t]["rp_2_epa_end"] for t in champ_team_nums}

print(team_to_epa[2056], team_to_rp_1_epa[2056], team_to_rp_2_epa[2056])

84.85 0.9931 0.5175


In [52]:
TOTAL_MEAN = 74.57
TOTAL_SD = 29.36

In [33]:
@lru_cache
def make_request(url):
    response = requests.get(url)
    response.raise_for_status()
    data = response.text
    lines = data.split("\n")
    return lines

@lru_cache
def get_schedule(num_teams: int, num_matches: int):
    # TODO: remove this once we have pre-generated schedules for 100+ teams
    if num_teams > 100:
        schedule1 = get_schedule(100, num_matches)
        schedule2 = get_schedule(num_teams - 100, num_matches)
        schedule2 = [
            {"red": [team + 100 for team in match["red"]], "blue": [team + 100 for team in match["blue"]]}
            for match in schedule2
        ]
        return schedule1 + schedule2

    # load csv from external URL using requests
    lines = make_request(f"https://raw.githubusercontent.com/Team254/cheesy-arena/main/schedules/{num_teams}_{num_matches}.csv")
    
    schedule = []
    for line in lines:
        match = line.split(",")
        if len(match) < 12:
            continue
        red = [int(match[0]), int(match[2]), int(match[4])]
        blue = [int(match[6]), int(match[8]), int(match[10])]
        schedule.append({"red": red, "blue": blue})
        
    return schedule

In [381]:
def get_win_prob(a, b):
    return 1 / (1 + 10 ** (((-5 / 8) * (a - b)) / TOTAL_SD))

def get_rp_pred(a):
    return 1 / (1 + np.e ** (-4 * (a - 0.5)))

def sim_single_quals(teams, schedule):
    curr_sim_matches = {t: 0 for t in teams}
    curr_sim_rps = {t: 0 for t in teams}
    for m in schedule:
        red_epa = sum(team_to_epa[x] for x in m["red"])
        blue_epa = sum(team_to_epa[x] for x in m["blue"])
        red_rp_1_epa = sum(team_to_rp_1_epa[x] for x in m["red"])
        blue_rp_1_epa = sum(team_to_rp_1_epa[x] for x in m["blue"])
        red_rp_2_epa = sum(team_to_rp_2_epa[x] for x in m["red"])
        blue_rp_2_epa = sum(team_to_rp_2_epa[x] for x in m["blue"])
        
        win_prob = get_win_prob(red_epa, blue_epa)
        red_win = 1 if random.random() < win_prob else 0

        red_rp_1_prob = get_rp_pred(0.9 * red_rp_1_epa)
        red_rp_1 = 1 if random.random() < red_rp_1_prob else 0
        
        red_rp_2_prob = get_rp_pred(red_rp_2_epa)
        red_rp_2 = 1 if random.random() < red_rp_2_prob else 0
        
        blue_rp_1_prob = get_rp_pred(0.9 * blue_rp_1_epa)
        blue_rp_1 = 1 if random.random() < blue_rp_1_prob else 0
        
        blue_rp_2_prob = get_rp_pred(blue_rp_2_epa)
        blue_rp_2 = 1 if random.random() < blue_rp_2_prob else 0

        red_rps = red_rp_1 + red_rp_2 + (2 if red_win else 0)
        blue_rps = blue_rp_1 + blue_rp_2 + (0 if red_win else 2)
        
        for x in m["red"]:
            curr_sim_matches[x] += 1
            if curr_sim_matches[x] <= 10:
                curr_sim_rps[x] += red_rps
            
        for x in m["blue"]:
            curr_sim_matches[x] += 1
            if curr_sim_matches[x] <= 10:
                curr_sim_rps[x] += blue_rps
            
    curr_sim_ranks = sorted(curr_sim_rps.items(), key=lambda x: [-x[1], random.random()])
    
    return [x[0] for x in curr_sim_ranks]

def softmax_select(options, mult=1):
    exp = [np.e ** (mult * o) for o in options]
    sum_exp = sum(exp)
    exp = [e / sum_exp for e in exp]
    rand = random.random()
    curr, i = 0, 0
    while curr < rand:
        curr += exp[i]
        i += 1
    return i - 1

def sim_alliance_selection(ranks):
    alliances = [[], [], [], [], [], [], [], []]
    locked_teams = []
    remaining_teams = [(r, team_to_epa[r]) for r in ranks]
    
    r1_mult = 1 / 3
    r2_mult = 1 / 2
    
    def handle_selection(selector, reject=True):
        selected = None
        while selected is None:
            temp_remaining_teams = [r for r in remaining_teams if r not in locked_teams]
            _selected = softmax_select([r[1] for r in temp_remaining_teams], r1_mult)
            _selected_team = temp_remaining_teams[_selected]
            orig_rank = ranks.index(_selected_team[0]) + 1
            if not reject or orig_rank > 8:
                selected = _selected
                continue
                
            selected_rank = remaining_teams.index(_selected_team) + 1
            remaining_best_epas = sorted(temp_remaining_teams[_selected + 1:], key=lambda x: -x[1])
            if remaining_best_epas[selected_rank][1] > selector[1] + 5:
                locked_teams.append(_selected_team)
            else:
                selected = _selected
                
        return remaining_teams.index(temp_remaining_teams[selected])
    
    # Captain and Round 1
    alliances[0].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[0].append(remaining_teams.pop(selected))
    
    alliances[1].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[1].append(remaining_teams.pop(selected))
    
    alliances[2].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[2].append(remaining_teams.pop(selected))
    
    alliances[3].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[3].append(remaining_teams.pop(selected))

    alliances[4].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[4].append(remaining_teams.pop(selected))
    
    alliances[5].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[5].append(remaining_teams.pop(selected))
    
    alliances[6].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[6].append(remaining_teams.pop(selected))
    
    alliances[7].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[7].append(remaining_teams.pop(selected))
    
    # Round 2
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[7].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[6].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[5].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[4].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[3].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[2].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[1].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[0].append(remaining_teams.pop(selected))
    
    # Round 3
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[0].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[1].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[2].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[3].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[4].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[5].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[6].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[7].append(remaining_teams.pop(selected))
    
    aepas = [sum(a[1] for a in alliance[:3]) for alliance in alliances]
    return [[a[0] for a in alliance] for alliance in alliances], aepas

def sim_single_elims(alliances):
    aepas = [sum(team_to_epa[a] for a in alliance[:3]) for alliance in alliances]
    ms = [[0, 7], [3, 4], [1, 6], [2, 5], [], [], [], [], [], [], [], [], []]
    
    def _get_win_prob(i):
        return get_win_prob(aepas[ms[i][0]], aepas[ms[i][1]])

    m1_red_winner = random.random() < _get_win_prob(0)
    ms[4].append(ms[0][1] if m1_red_winner else ms[0][0])
    ms[6].append(ms[0][0] if m1_red_winner else ms[0][1])
    
    m2_red_winner = random.random() < _get_win_prob(1)
    ms[4].append(ms[1][1] if m2_red_winner else ms[1][0])
    ms[6].append(ms[1][0] if m2_red_winner else ms[1][1])
    
    m3_red_winner = random.random() < _get_win_prob(2)
    ms[5].append(ms[2][1] if m3_red_winner else ms[2][0])
    ms[7].append(ms[2][0] if m3_red_winner else ms[2][1])
    
    m4_red_winner = random.random() < _get_win_prob(3)
    ms[5].append(ms[3][1] if m4_red_winner else ms[3][0])
    ms[7].append(ms[3][0] if m4_red_winner else ms[3][1])
    
    m5_red_winner = random.random() < _get_win_prob(4)
    ms[9].append(ms[4][0] if m5_red_winner else ms[4][1])
    
    m6_red_winner = random.random() < _get_win_prob(5)
    ms[8].append(ms[5][0] if m6_red_winner else ms[5][1])
    
    m7_red_winner = random.random() < _get_win_prob(6)
    ms[8].append(ms[6][1] if m7_red_winner else ms[6][0])
    ms[10].append(ms[6][0] if m7_red_winner else ms[6][1])
    
    m8_red_winner = random.random() < _get_win_prob(7)
    ms[9].append(ms[7][1] if m8_red_winner else ms[7][0])
    ms[10].append(ms[7][0] if m8_red_winner else ms[7][1])
    
    m9_red_winner = random.random() < _get_win_prob(8)
    ms[11].append(ms[8][0] if m9_red_winner else ms[8][1])
    
    m10_red_winner = random.random() < _get_win_prob(9)
    ms[11].append(ms[9][0] if m10_red_winner else ms[9][1])
    
    m11_red_winner = random.random() < _get_win_prob(10)
    ms[12].append(ms[10][1] if m11_red_winner else ms[10][0])
    finalist_1 = ms[10][0] if m11_red_winner else ms[10][1]
    
    m12_red_winner = random.random() < _get_win_prob(11)
    fourth_place = ms[11][1] if m12_red_winner else ms[11][0]
    ms[12].append(ms[11][0] if m12_red_winner else ms[11][1])
    
    m13_red_winner = random.random() < _get_win_prob(12)
    third_place = ms[12][1] if m13_red_winner else ms[12][0]
    finalist_2 = ms[12][0] if m13_red_winner else ms[12][1]
    
    
    f1_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    f2_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    f3_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    winner = finalist_1 if f1_red_winner + f2_red_winner + f3_red_winner >= 2 else finalist_2
    second_place = finalist_2 if winner == finalist_1 else finalist_1
    
    return winner, second_place, third_place, fourth_place


In [383]:
overall_winner_count = defaultdict(int)
overall_finalist_count = defaultdict(int)
overall_third_place_count = defaultdict(int)
overall_fourth_place_count = defaultdict(int)

winner_count = defaultdict(int)
finalist_count = defaultdict(int)
third_place_count = defaultdict(int)
fourth_place_count = defaultdict(int)

winner_alliances = defaultdict(int)
winner_best_epas = []
alliance_epas = []
best_epas = []
winner_epas = []

einstein_winner_best_epas = []
einstein_alliance_epas = []
einstein_best_epas = []
einstein_winner_epas = []

qual_rank_list = defaultdict(lambda: [0] * 78)

num_sims = 100000

def pre_sim(all_teams):
    for sim in range(num_sims):
        if sim % 100 == 0:
            print(sim)
            
        random.shuffle(all_teams)

        division_1 = all_teams[:78]
        division_2 = all_teams[78:156]
        division_3 = all_teams[156:233]
        division_4 = all_teams[233:310]
        division_5 = all_teams[310:388]
        division_6 = all_teams[388:466]
        division_7 = all_teams[466:543]
        division_8 = all_teams[543:620]

        divisions = [division_1, division_2, division_3, division_4, division_5, division_6, division_7, division_8]

        einstein_alliances = []
        einstein_aepas = []
        for division in divisions:
            random.shuffle(division)
            schedule = get_schedule(len(division), 10)
            schedule = [
                {
                    "red": [division[x - 1] for x in m["red"]],
                    "blue": [division[x - 1] for x in m["blue"]],
                }
                for m in schedule
            ]
            qual_ranks = sim_single_quals(division, schedule)
            for i, t in enumerate(qual_ranks):
                qual_rank_list[t][i] += 1
            
            alliances, aepas = sim_alliance_selection(qual_ranks)
            winner, finalist, third_place, fourth_place = sim_single_elims(alliances)
            einstein_alliances.append(alliances[winner])
            einstein_aepas.append(aepas[winner])
            
            winner_alliances[winner] += 1
            best_epa = max(aepas)
            winner_epa = aepas[winner]
            winner_best_epas.append(best_epa == winner_epa)
            alliance_epas.extend(aepas)
            best_epas.append(best_epa)
            winner_epas.append(winner_epa)
            
            for t in alliances[winner]:
                winner_count[t] += 1
                
            for t in alliances[finalist]:
                finalist_count[t] += 1
                
            for t in alliances[third_place]:
                third_place_count[t] += 1
                
            for t in alliances[fourth_place]:
                fourth_place_count[t] += 1
            
        winner, finalist, third_place, fourth_place = sim_single_elims(einstein_alliances)

        einstein_best_epa = max(einstein_aepas)
        einstein_winner_epa = einstein_aepas[winner]
        einstein_winner_best_epas.append(einstein_best_epa == einstein_winner_epa)
        einstein_alliance_epas.extend(einstein_aepas)
        einstein_best_epas.append(einstein_best_epa)
        einstein_winner_epas.append(einstein_winner_epa)

        for t in einstein_alliances[winner]:
            overall_winner_count[t] += 1
            
        for t in einstein_alliances[finalist]:
            overall_finalist_count[t] += 1
            
        for t in einstein_alliances[third_place]:
            overall_third_place_count[t] += 1
            
        for t in einstein_alliances[fourth_place]:
            overall_fourth_place_count[t] += 1
            
pre_sim(champ_team_nums)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [406]:
def pad(s, length = 15):
    return s + "&nbsp;" * (length - len(s))

print("Division")
print("| Rank | ", pad("Team"), " | ", pad("Winner"), " | ", pad("Finalist"), " | ", pad("3rd Place"), " | ", pad("4th Place"), " |")
print("| ---- | ---- | --- | --- | --- | --- |")
for i, (key, count) in enumerate(sorted(winner_count.items(), key=lambda x: -x[1])[:25]):
    print("|", i + 1, "|", key, "|", str(round(100 * winner_count[key] / num_sims, 1)) + "%", "|", str(round(100 * finalist_count[key] / num_sims, 1)) + "%", "|", str(round(100 * third_place_count[key] / num_sims, 1)) + "%", "|", str(round(100 * fourth_place_count[key] / num_sims, 1)) + "%", "|")
print()

print("Division")
print("| Rank | ", pad("Team"), " | ", pad("Winner"), " | ", pad("Finalist"), " | ", pad("3rd Place"), " | ", pad("4th Place"), " |")
print("| ---- | ---- | --- | --- | --- | --- |")
for i, (key, count) in enumerate(sorted(winner_count.items(), key=lambda x: -x[1])[:100]):
    print("|", i + 1, "|", key, "|", str(round(100 * winner_count[key] / num_sims, 1)) + "%", "|", str(round(100 * finalist_count[key] / num_sims, 1)) + "%", "|", str(round(100 * third_place_count[key] / num_sims, 1)) + "%", "|", str(round(100 * fourth_place_count[key] / num_sims, 1)) + "%", "|")
print()

print("Einstein")
print("| Rank | ", pad("Team"), " | ", pad("Winner"), " | ", pad("Finalist"), " | ", pad("3rd Place"), " | ", pad("4th Place"), " |")
print("| ---- | ---- | --- | --- | --- | --- |")
for i, (key, count) in enumerate(sorted(overall_winner_count.items(), key=lambda x: -x[1])[:25]):
    print("|", i + 1, "|", key, "|", str(round(100 * overall_winner_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_finalist_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_third_place_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_fourth_place_count[key] / num_sims, 1)) + "%", "|")
print()

print("Einstein")
print("| Rank | ", pad("Team"), " | ", pad("Winner"), " | ", pad("Finalist"), " | ", pad("3rd Place"), " | ", pad("4th Place"), " |")
print("| ---- | ---- | --- | --- | --- | --- |")
for i, (key, count) in enumerate(sorted(overall_winner_count.items(), key=lambda x: -x[1])[:100]):
    print("|", i + 1, "|", key, "|", str(round(100 * overall_winner_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_finalist_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_third_place_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_fourth_place_count[key] / num_sims, 1)) + "%", "|")
print()

print("| Alliance | Win Probability |")
print("| --- | --- |")
for i, count in sorted(winner_alliances.items(), key=lambda x: -x[1]):
    print("|", i + 1, "|", str(round(100 * count / 8 / num_sims, 2)) + "%", "|")
print()

print("Probability 'Favorite' Wins:", round(np.array(winner_best_epas).mean(), 4))
print("Average Division Alliance EPA:", round(np.array(alliance_epas).mean(), 1))
print("Average Division 'Favorite' EPA:", round(np.array(best_epas).mean(), 1))
print("Average Division Winner EPA:", round(np.array(winner_epas).mean(), 1))
print()

print("Probability Einstein 'Favorite' Wins:", round(np.array(einstein_winner_best_epas).mean(), 4))
print("Average Einstein Alliance EPA:", round(np.array(einstein_alliance_epas).mean(), 1))
print("Average Einstein 'Favorite' EPA:", round(np.array(einstein_best_epas).mean(), 1))
print("Average Einstein Winner EPA:", round(np.array(einstein_winner_epas).mean(), 1))
print()

Division
| Rank |  Team&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Winner&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Finalist&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  3rd Place&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  4th Place&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |
| ---- | ---- | --- | --- | --- | --- |
| 1 | 2056 | 67.4% | 11.7% | 7.0% | 4.8% |
| 2 | 1323 | 58.5% | 14.0% | 9.0% | 6.0% |
| 3 | 254 | 58.0% | 14.3% | 9.0% | 6.1% |
| 4 | 1678 | 41.5% | 16.9% | 11.9% | 8.8% |
| 5 | 3005 | 38.6% | 17.1% | 12.4% | 9.2% |
| 6 | 111 | 32.0% | 17.5% | 13.2% | 10.3% |
| 7 | 971 | 31.7% | 17.3% | 13.1% | 10.5% |
| 8 | 3538 | 30.9% | 17.5% | 13.3% | 10.5% |
| 9 | 5940 | 30.2% | 17.6% | 13.3% | 10.6% |
| 10 | 6036 | 29.5% | 17.5% | 13.5% | 10.4% |
| 11 | 930 | 25.0% | 17.2% | 13.9% | 11.3% |
| 12 | 6672 | 24.5% | 17.0% | 13.8% | 11.5% |
| 13 | 7157 | 23.8% | 16.9% | 13.9% | 11.5% |
| 14 | 1987 | 21.1% | 16.5% | 13.9% | 12.1% |
| 15 | 1619 | 20.9% | 16

Average Einstein Alliance EPA: 180.9
Average Einstein 'Favorite' EPA: 202.6
Average Einstein Winner EPA: 196.4



In [405]:
avg_ranks = {t: sum(((i + 1) * qual_rank_list[t][i]) for i in range(78)) / num_sims for t in qual_rank_list}
top_1 = {t: sum(qual_rank_list[t][i] for i in range(1)) / num_sims for t in qual_rank_list}
top_4 = {t: sum(qual_rank_list[t][i] for i in range(4)) / num_sims for t in qual_rank_list}
top_8 = {t: sum(qual_rank_list[t][i] for i in range(8)) / num_sims for t in qual_rank_list}
top_16 = {t: sum(qual_rank_list[t][i] for i in range(16)) / num_sims for t in qual_rank_list}

print("| Rank | ", pad("Team"), " | ", pad("Mean Rank"), " | ", pad("1st"), " | ", pad("Top 4"), " | ", pad("Top 8"), " | ", pad("Top 16"), " |")
print("| --- | --- | --- | --- | --- | --- | --- |")
for i, (key, avg_rank) in enumerate(sorted(avg_ranks.items(), key=lambda x: x[1])[:25]):
    print("|", i + 1, "|", key, "|", round(avg_rank, 2), "|", str(round(100 * top_1[key], 1)) + "%", "|", str(round(100 * top_4[key], 1)) + "%", "|", str(round(100 * top_8[key], 1)) + "%", "|", str(round(100 * top_16[key], 1)) + "%", "|")
print()
    
print("| Rank | ", pad("Team"), " | ", pad("Mean Rank"), " | ", pad("1st"), " | ", pad("Top 4"), " | ", pad("Top 8"), " | ", pad("Top 16"), " |")
print("| --- | --- | --- | --- | --- | --- | --- |")
for i, (key, avg_rank) in enumerate(sorted(avg_ranks.items(), key=lambda x: x[1])[:100]):
    print("|", i + 1, "|", key, "|", round(avg_rank, 2), "|", str(round(100 * top_1[key], 1)) + "%", "|", str(round(100 * top_4[key], 1)) + "%", "|", str(round(100 * top_8[key], 1)) + "%", "|", str(round(100 * top_16[key], 1)) + "%", "|")
print()


| Rank |  Team&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Mean Rank&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  1st&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 8&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 16&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | 2056 | 5.32 | 29.1% | 63.4% | 81.1% | 93.8% |
| 2 | 1323 | 5.68 | 27.1% | 60.7% | 79.0% | 92.9% |
| 3 | 254 | 6.09 | 25.4% | 58.4% | 77.0% | 91.7% |
| 4 | 1678 | 7.68 | 19.0% | 48.6% | 68.8% | 87.3% |
| 5 | 3538 | 9.87 | 13.3% | 38.5% | 58.8% | 80.5% |
| 6 | 971 | 11.01 | 11.2% | 34.5% | 53.8% | 76.6% |
| 7 | 930 | 11.12 | 10.8% | 33.5% | 53.2% | 76.3% |
| 8 | 3005 | 11.19 | 10.9% | 33.9% | 53.3% | 76.0% |
| 9 | 6036 | 11.21 | 10.7% | 33.4% | 53.2% | 76.2% |
| 10 | 111 | 11.36 | 10.6% | 33.1% | 52.5% | 75.6% |
| 11 | 1987 | 1

In [ ]:
# Doesn't account for: 
# supercharing, 
# new link RP, 
# alliance synergy / second pick defense / scorched field
# team improvement, 

